In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns
from pathlib import Path
sns.set_style('whitegrid')  # Set whitegrid style from Seaborn
#import mplfinance as mpf
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import matplotlib.ticker as tkr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from collections import defaultdict
from scipy import stats
from datetime import datetime
import pytz
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import neptune as neptune

In [9]:
df  = pd.read_csv('provider 1 GR price forecasts 20240327-20240415.csv')

In [10]:
# Convert datetime columns to datetime type for easier manipulation
df['datetime_from'] = pd.to_datetime(df['datetime_from'])
df['issue_date'] = pd.to_datetime(df['issue_date'])

# Sort by datetime_from and issue_date to get the latest issue_date for each hour
df_sorted = df.sort_values(by=['datetime_from', 'issue_date'], ascending=[True, False])

# Drop duplicates to keep only the latest issue_date for each hour
df_unique = df_sorted.drop_duplicates(subset=['datetime_from'], keep='first')

In [12]:
df_final = df_unique[['datetime_from', 'value']]

In [13]:
df_final

,datetime_from,value
0,2024-03-27 08:00:00+01:00,74.99
1,2024-03-27 09:00:00+01:00,62.19
2,2024-03-27 10:00:00+01:00,52.03
3,2024-03-27 11:00:00+01:00,45.00
4,2024-03-27 12:00:00+01:00,42.96
5,2024-03-27 13:00:00+01:00,43.56
6,2024-03-27 14:00:00+01:00,46.74
7,2024-03-27 15:00:00+01:00,61.11
8,2024-03-27 16:00:00+01:00,74.73
9,2024-03-27 17:00:00+01:00,82.50


In [14]:
df_final.to_csv('provider 1 GR price forecasts extra.csv')